In [3]:
import numpy as np
import glob
import os,sys
import pandas as pd
import re
curr_path = sys.path[0]
os.chdir(curr_path+"/../Analytic_Flux/ProcaEnDenCSVData")
curr_path = os.getcwd()+"/"

def contains_str(str, regex):
    _regex = re.compile(regex)
    return bool(_regex.search(str))

In [4]:
allfiles = glob.glob(curr_path+"*")
allfilenames = np.array(list(map(os.path.basename, allfiles)))
fun = lambda str: contains_str(str, "EnergyDensityCOORDS*")
COORDS_filenames = allfilenames[list(map(fun, allfilenames))]
fun = lambda str: contains_str(str, "EnergyDensityVALUES*")
VALUES_filenames = allfilenames[list(map(fun, allfilenames))]


In [18]:
coordvaluepair = []
for valuestr in VALUES_filenames:
    try:
        data_name = valuestr[20:]
        truth = [i[20:]==valuestr[20:] for i in COORDS_filenames]
        coordstr = COORDS_filenames[truth][0]
        coord_data = pd.read_csv(coordstr, header=None, na_values="None")
        value_data = pd.read_csv(valuestr, header=None, na_values="None")
        rdata = coord_data[0].dropna().values
        thdata = coord_data[1].dropna().values
        valuedata = value_data.values

        out_filename = curr_path+"../ProcaEnDen/"+data_name[:-4]
        np.savez(out_filename, RadialData=rdata, ThetaData=thdata, EnergyData=valuedata)
    except KeyError:
        print("Error at: \n\t {0} \n\t {1}\n skipping \n".format(valuestr,coordstr))

Error at: 
	 EnergyDensityVALUES_BHSpin_9_10_Alpha_101_100_Mode_3_Overtone_0.dat 
	 EnergyDensityCOORDS_BHSpin_9_10_Alpha_101_100_Mode_3_Overtone_0.dat
 skipping 

Error at: 
	 EnergyDensityVALUES_BHSpin_9_10_Alpha_127_200_Mode_2_Overtone_4.dat 
	 EnergyDensityCOORDS_BHSpin_9_10_Alpha_127_200_Mode_2_Overtone_4.dat
 skipping 



In [19]:
alldata = np.array(glob.glob(curr_path+"../ProcaEnDen/*"))

In [20]:
alldata
modeovertonedata = alldata[[i[-21:-4]=="Mode_"+str(1)+"_Overtone_"+str(0) for i in alldata]]
alpha_regex= re.compile("Alpha_\d+_\d+")
alpha_matches = np.array([alpha_regex.findall(string) for string in modeovertonedata])
value_regex = re.compile("\d+")
numdoms = [list(map(float, value_regex.findall(st[0]))) for st in alpha_matches]
alphas=np.array([i[0]/i[1] for i in numdoms])
tmp =np.abs(0.5-alphas)
index = np.where(tmp==tmp.min())
result = np.array(modeovertonedata)[index][0]

In [21]:
data=np.load(result)

In [26]:
data["EnergyData"].shape

(310, 100)

In [27]:
data["RadialData"].shape

(310,)

In [28]:
data["ThetaData"].shape

(100,)

In [30]:
alphas.min()

0.06